<a href="https://colab.research.google.com/github/wel51x/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module2-sql-for-analysis/LS_DS2_322_PostgreSQL_Connection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Live demo code for connecting to PostgreSQL from Python.

In [0]:
!pip install psycopg2-binary

In [0]:
import psycopg2

In [0]:
dbname = 'fxbzafim'
user = 'fxbzafim'
password = 'wqVYFX8ryOe6QXiUzYk_kbJj8SnRJ6nv'  # Don't commit!
host = 'isilo.db.elephantsql.com'

In [0]:
pg_conn = psycopg2.connect(dbname=dbname, user=user,
                           password=password, host=host)

In [0]:
pg_curs = pg_conn.cursor()

In [0]:
pg_curs.execute('SELECT * FROM test_table;')
pg_curs.fetchall()

[(1, 'A row name', None),
 (2, 'Another row, with JSON', {'a': 1, 'b': ['dog', 'cat', 42], 'c': True})]

In [0]:
!wget https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true

--2019-03-26 16:40:23--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/blob/master/module1-introduction-to-sql/rpg_db.sqlite3?raw=true
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-03-26 16:40:23--  https://github.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/raw/master/module1-introduction-to-sql/rpg_db.sqlite3
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databases/master/module1-introduction-to-sql/rpg_db.sqlite3 [following]
--2019-03-26 16:40:23--  https://raw.githubusercontent.com/LambdaSchool/DS-Unit-3-Sprint-2-SQL-and-Databas

In [0]:
!mv 'rpg_db.sqlite3?raw=true' rpg_db.sqlite3

In [0]:
import sqlite3
sl_conn = sqlite3.connect('rpg_db.sqlite3')
sl_curs = sl_conn.cursor()

In [0]:
sl_curs.execute('SELECT COUNT(*) FROM charactercreator_character;').fetchall()

[(302,)]

In [0]:
# Our goal - an ETL/data pipeline
# Get the character data from SQLite to PostgreSQL!
characters = sl_curs.execute('SELECT * FROM charactercreator_character;').fetchall()

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
characters[-1]

(302, 'Aliquam n', 0, 0, 10, 1, 1, 1, 1)

In [0]:
# Character schema for PostgreSQL
create_character_table = """
CREATE TABLE charactercreator_character (
  character_id SERIAL PRIMARY KEY,
  name VARCHAR(30),
  level INT,
  exp INT,
  hp INT,
  strength INT,
  intelligence INT,
  dexterity INT,
  wisdom INT
)
"""

In [0]:
pg_curs.execute(create_character_table)

In [0]:
characters[0][1:]

('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
"""
  INSERT INTO charactercreator_character
  (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
  VALUES""" + str(characters[0][1:])

"\n  INSERT INTO charactercreator_character\n  (name, level, exp, hp, strength, intelligence, dexterity, wisdom)\n  VALUES('Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)"

In [0]:
# Loop over and actually insert all the results
for character in characters:
  insert_character = """
  INSERT INTO charactercreator_character
  (name, level, exp, hp, strength, intelligence, dexterity, wisdom)
  VALUES """ + str(character[1:])
  pg_curs.execute(insert_character)

In [0]:
pg_conn.commit()

In [0]:
# Let's remake the pg cursor and check things
pg_curs = pg_conn.cursor()
pg_curs.execute('SELECT * FROM charactercreator_character;')
pg_characters = pg_curs.fetchall()

In [0]:
characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
pg_characters[0]

(1, 'Aliquid iste optio reiciendi', 0, 0, 10, 1, 1, 1, 1)

In [0]:
for character, pg_character in zip(characters, pg_characters):
  assert character == pg_character

In [0]:
# Nothing happened - which means they're all the same!